In [ ]:
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from nba_api.stats.endpoints.shotchartdetail import ShotChartDetail
from tqdm import tqdm

data_DIR = "../data/"
export_DIR = "../fdata/ShotLocationData/"

In [ ]:
# Only 1 season
year = 2023
league, league_id, season = "NBA", "00", str(year) + "-" + str(year + 1)[-2:]
data = pd.read_csv(data_DIR + f"{league}_players_database.csv")
player_data = data[(data["From"] <= year) & (data["To"] >= year)]
player_data = player_data.reset_index(drop=True)
player_dict = player_data.to_dict(orient="records")
player_ids = [player["pID"] for player in player_dict]
len(player_ids)

In [ ]:
shota = []
for player_id in tqdm(player_ids):
    for ii in range(5):
        # time.sleep(0.6)
        try:
            player_shotchart = ShotChartDetail(
                league_id=league_id,
                team_id=0,
                player_id=player_id,
                context_measure_simple="FGA",
                season_nullable=season,
            )
            shots_df = player_shotchart.get_data_frames()[0]
            shots_df["LOC_X"] = -shots_df["LOC_X"]
            shota.append(shots_df)
            break
        except Exception as error:
            print(player_id)
            print(error)
            time.sleep(0.6)
            continue
shots = pd.concat(shota)
# shots.to_csv(export_DIR+ league + f"_Shot_Details_{year}.csv", index=False)
shots.to_parquet(export_DIR+ league + f"_Shot_Details_{year}.parquet")